In [ ]:
# logウィジェット作成
from ipywidgets.widgets import Textarea
from ipywidgets import Layout
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label

class LogWidget(Textarea):
    log_no = 0

    def __init__(self, *args, **kwargs):
        super(Textarea, self).__init__(*args, **kwargs)

        """
        set default description and style
        """
        self.description = 'log'
        self.style = {'description_width': 'initial'}
        self.maxline = 0 # no limit
        self.layout = Layout(width='100%')
        """
        update from arguments
        vのvalidationはしない。理由は適切な範囲を考えるのがめんどくさそうだから。
        """
        for k, v in kwargs.items():
            if k == 'description':
                self.description = v
            elif k == 'style':
                self.style = v
            elif k == 'maxline':
                self.maxline = v
            elif k == 'layout':
                self.layout = v


    def __call__(self, msg):
        """
        write log
        """
        self.log_no = self.log_no + 1
        if self.maxline > 0:
            # 最新のメッセージ + 過去maxline行分を表示します。
            self.value = str(self.log_no) + ": " + str(msg) + "\n" + "\n".join(self.value.splitlines()[:self.maxline])
        else:
            self.value = str(self.log_no) + ": " + str(msg) + "\n" + self.value


In [ ]:
from donkeycar.parts.keras import KerasLinear

pilot = KerasLinear()
# モデルをロード
pilot.load("linear.h5")


In [ ]:
import cv2

# 画像を読み込み、適切な形状に変換
image = cv2.imread("data/images/1_cam_image_array_.jpg")

# 推論を実行
output = pilot.run(image)

print("推論結果:", output)

In [ ]:
from ipyevents import Event 
import PIL.Image
import io
import os
import json
import orjson
import re
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
import time
import mmap
import multiprocessing
from multiprocessing import Pool
import logging
import copy
from collections import defaultdict
import threading

write_log = LogWidget(maxline=20)

directory = 'data'

# これらの関数はトップレベルで定義する必要があります
def process_manifest(args):
    directory, n_catalog_files, deleted_indexes = args
    file_deleted_indexes = []
    for catalog_file in n_catalog_files:
        filepath = os.path.join(directory, catalog_file)
        with open(filepath, 'r') as file:
            for line in file:
                record = orjson.loads(line)
                if record['_index'] in deleted_indexes:
                    file_deleted_indexes.append(record['_index'])
    return {catalog_file: file_deleted_indexes}
                
def process_refresh(args):
    directory, n_catalog_files, deleted_indexes = args  # 引数を展開
    annotations = []
    annotations_deleted = []
    index_to_catalog = {}

    for catalog_file in n_catalog_files:
        filepath = os.path.join(directory, catalog_file)
        catalog_deleted_indexes = deleted_indexes[catalog_file]
        with open(filepath, 'r') as file:
            for line in file:
                record = orjson.loads(line)
                record_info = {"catalog_file": catalog_file}
                record_info.update(record)
                if record['_index'] in catalog_deleted_indexes:
                    annotations_deleted.append(record_info)
                else:
                    annotations.append(record_info)
                    index_to_catalog[record['_index']] = catalog_file
    return annotations, annotations_deleted, index_to_catalog  # 処理結果を返します

def process_save_record(args):
    filepath, start_line, end_line, index, record_info = args
    # {"_index": 0, "_session_id": "23-11-29_0", "_timestamp_ms": 1701255880346, "cam/image_array": "0_cam_image_array_.jpg", "user/angle": 0.0, "user/mode": "user", "user/throttle": 0.028565324869533372}
    # {"_index":0, "_session_id": "23-11-29_0", "_timestamp_ms": 1701255880346, "cam/image_array": "0_cam_image_array_.jpg", "user/angle": 0.0, "user/mode": "user", "user/throttle": 0.028565324869533372}

    #new_record_str = orjson.dumps(record_info).decode('utf-8')  # orjsonは最適化されるため空白が消える。
    new_record_str = json.dumps(record_info, indent=None, separators=(', ', ': '))
    pattern = re.compile(r'({.*"_index":\s*%s\s*,.*})' % index)
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()[start_line:end_line]
    
    updated_lines = [pattern.sub(new_record_str, line) if pattern.search(line) else line for line in lines]

    return start_line, updated_lines

def update_catalog_manifest(manifest_file, index, new_length):
    if not os.path.exists(manifest_file):
        # Handle error or create a new manifest file
        return

    with open(manifest_file, 'r+') as file:
        manifest_data = json.load(file)
        line_lengths = manifest_data.get("line_lengths", [])
        
        if index < len(line_lengths):
            line_lengths[index] = new_length
        else:
            # Handle error: the index is out of range
            return

        manifest_data["line_lengths"] = line_lengths
        file.seek(0)
        file.truncate()
        json.dump(manifest_data, file)

class DonkeyDataset:
    def __init__(self, directory):
        self.directory = directory
        self.load_manifest()
        self.refresh()

    def __len__(self):
        return len(self.annotations)

    def load_manifest(self, one_process_one_catalog=True):
        """
        manifest.jsonを読み込み、catalogファイル毎にdeleted_indexesを保持する
        {
            "deleted_indexes": {
                "catalog_0.catalog": [0, 1, 2],
                "catalog_1.catalog": [1000, 1001, 1002],
                ...
            },
            ...
        }
        """
        self.deleted_indexes = defaultdict(list)

        with open(os.path.join(self.directory, 'manifest.json'), 'r') as file:
            manifest_lines = file.readlines()
            deleted_indexes = orjson.loads(manifest_lines[4])['deleted_indexes']

        files = [f for f in os.listdir(self.directory) if f.startswith('catalog_') and f.endswith('.catalog')]
        sorted_files = sorted(files, key=lambda x: int(re.search(r'(\d+)', x).group(1)))

        if one_process_one_catalog:
            # catalogファイルの数だけサブプロセスを作成します
            args = [(self.directory, [catalog_file], deleted_indexes) for catalog_file in sorted_files]
        else:
            # CPUコア数だけサブプロセスを作成します
            quarter_length = len(sorted_files) // multiprocessing.cpu_count()
            catalog_files = [sorted_files[i:i + quarter_length] for i in range(0, len(sorted_files), quarter_length)]
            args = [(self.directory, n_catalog_files, deleted_indexes) for n_catalog_files in catalog_files]

        with multiprocessing.Pool() as pool:
            results = pool.map(process_manifest, args)

        for result in results:
            self.deleted_indexes.update(result)
    
    def refresh(self, one_process_one_catalog=True):
        #self.load_manifest()
        self.annotations = []
        self.annotations_deleted = []
        self.index_to_catalog = {}
        files = [f for f in os.listdir(self.directory) if f.startswith('catalog_') and f.endswith('.catalog')]
        sorted_files = sorted(files, key=lambda x: int(re.search(r'(\d+)', x).group(1)))

        if one_process_one_catalog:
            # catalogファイルの数だけサブプロセスを作成します
            args = [(self.directory, [catalog_file], self.deleted_indexes) for catalog_file in sorted_files]
        else:
            # CPUコア数だけサブプロセスを作成します
            quarter_length = len(sorted_files) // multiprocessing.cpu_count()
            catalog_files = [sorted_files[i:i + quarter_length] for i in range(0, len(sorted_files), quarter_length)]
            args = [(self.directory, n_catalog_files, self.deleted_indexes) for n_catalog_files in catalog_files]
        
        # 各ファイルサブセットに対してprocess_refreshを呼び出します
        with multiprocessing.Pool() as pool:
            results = pool.map(process_refresh, args)
    
        # resultsからannotations, annotations_deleted, index_to_catalogを抽出し結合
        for annotations, annotations_deleted, index_to_catalog in results:
            self.annotations.extend(annotations)
            self.annotations_deleted.extend(annotations_deleted)
            self.index_to_catalog.update(index_to_catalog)

    def save_record(self, index, record_info):
        annotation = copy.copy(record_info)
        # Step 1: catalog_fileを読み込み、recordから削除
        # record_infoからcatalog_fileキーを削除
        catalog_file = record_info.pop('catalog_file', None)
        index = record_info["_index"]
        filepath = os.path.join(self.directory, catalog_file)
        # ファイルに書き込む
        self.save_record_parallel((filepath, index, record_info))
        # annotationsも更新する
        #self.refresh()
        self.update_annotation(annotation)

    def update_annotation(self, new_annotation):
        # annotationsを更新する
        index = new_annotation["_index"]
        for annotation in self.annotations:
            if annotation["_index"] == index:
                # Update the annotation
                annotation.update(new_annotation)
                break

    def save_record_parallel(self, args):
        filepath, index, record_info = args
        total_lines = sum(1 for line in open(filepath, 'r', encoding='utf-8'))

        half = total_lines // 2
        with multiprocessing.Pool() as pool:
            updated_segments = pool.map(process_save_record, [
                (filepath, 0, half, index, record_info),
                (filepath, half, total_lines, index, record_info)
            ])
            
        # 更新されたセグメントを結合し、ファイル全体の内容を再構築します
        updated_content = []
        for start_line, updated_lines in sorted(updated_segments, key=lambda x: x[0]):  # 開始行でソート
            updated_content.extend(updated_lines)

        # 更新された内容をファイルに書き戻します
        with open(filepath, 'w', encoding='utf-8') as file:
            file.writelines(updated_content)

        # .catalog_manifestファイルに書き込むためのjson文字列のバイト値を計算します
        new_record_str = json.dumps(record_info, indent=None, separators=(', ', ': '))
        new_length = len(new_record_str) + 1  # +1 for the newline character
        # .catalog_manifestファイルのバイト値を更新します。
        manifest_file = filepath + "_manifest"
        update_catalog_manifest(manifest_file, index, new_length)
        write_log(f'{manifest_file}, {index}, {new_length}')
    
    def delete_record(self, annotation_id):
        manifest_path = os.path.join(self.directory, 'manifest.json')        
        # Step 1: manifest.jsonを読み込む
        with open(manifest_path, 'r') as file:
            manifest_lines = file.readlines()
        
        # JSONオブジェクト作成する
        manifest_obj = orjson.loads(manifest_lines[4])
        
        # Step 2: deleted_indexesリストを取得し、指定されたindexが既にリストに存在するかどうかを確認する
        deleted_indexes = manifest_obj.get('deleted_indexes', [])
        record_info = self.annotations.pop(annotation_id)
        index = record_info["_index"]
        if index in deleted_indexes:
            write_log(f'Index {index} is already in deleted_indexes.')
            return  # indexが既に存在する場合、関数を終了する
        
        # Step 3: deleted_indexesリストにindexを追加する
        deleted_indexes.append(index)
        manifest_obj['deleted_indexes'] = deleted_indexes
        manifest_lines[4] = orjson.dumps(manifest_obj).decode('utf-8') + '\n'
        
        # Step 4: 更新されたdeleted_indexesリストをmanifest.jsonファイルに書き戻す
        with open(manifest_path, 'w') as file:
            file.writelines(manifest_lines)
        
        # deleted_indexesを保持するmanifestを再読込する
        #self.load_manifest()  # ファイルから読み込んで全体更新すると処理が遅いので、削除されたレコードの配列情報を更新する
        catalog_file = record_info["catalog_file"]
        if catalog_file in self.deleted_indexes and index in self.deleted_indexes[catalog_file]:
            write_log(f'Index {index} is already in self.deleted_indexes for {catalog_file}.')
            return  # indexが既に存在する場合、関数を終了する
        
        self.deleted_indexes[catalog_file].append(index)
        self.annotations_deleted.append(record_info)


# 使用例
dataset = DonkeyDataset(directory)
# これで、dataset.annotationsおよびdataset.annotations_deletedはそれぞれのデータエントリと関連するカタログファイル名を持っています。

In [ ]:
########################################
# remap
########################################
def remap(x, oMin, oMax, nMin, nMax):
    """
    args:
      x: input value
      oMin: old minimum value
      oMax: old maximum value
      nMin: new minimum value
      nMax: new maximum value
    """

    #range check
    if oMin == oMax:
        print("Warning: Zero input range")
        return None

    if nMin == nMax:
        print("Warning: Zero output range")
        return None

    #check reversed input range
    reverseInput = False
    oldMin = min( oMin, oMax )
    oldMax = max( oMin, oMax )
    if not oldMin == oMin:
        reverseInput = True

    #check reversed output range
    reverseOutput = False   
    newMin = min( nMin, nMax )
    newMax = max( nMin, nMax )
    if not newMin == nMin :
        reverseOutput = True

    portion = (x-oldMin)*(newMax-newMin)/(oldMax-oldMin)
    if reverseInput:
        portion = (oldMax-x)*(newMax-newMin)/(oldMax-oldMin)

    result = portion + newMin
    if reverseOutput:
        result = newMax - portion

    return result


In [ ]:
import cv2
import numpy as np

control_image_width = 250
control_image_height = 250
def draw_font(image, message, font, font_color):
    W, H = image.size
    draw = ImageDraw.Draw(image)
    _, _, w, h = draw.textbbox((0, 0), message, font=font)
    draw.text(((W-w)/2, (H-h)/2), message, font=font, fill=font_color)
    return image

def draw_cross_line(image, coordinates, color, width=2):
    # ImageDrawオブジェクトを作成
    draw = ImageDraw.Draw(image)
    
    # 垂直と水平の直線を描く
    draw.line([(coordinates[0], 0), (coordinates[0], control_image_height)], fill=color[0], width=width)
    draw.line([(0, coordinates[1]), (control_image_width, coordinates[1])], fill=color[1], width=width)
    
    return image  # 更新された画像を返す

def draw_st_cross_line(image, coordinates, color, width=2):
    """
    steeringは縦の線を最後に描く
    """
    # ImageDrawオブジェクトを作成
    draw = ImageDraw.Draw(image)
    
    # 垂直と水平の直線を描く
    draw.line([(0, coordinates[1]), (control_image_width, coordinates[1])], fill=color[1], width=width)
    draw.line([(coordinates[0], 0), (coordinates[0], control_image_height)], fill=color[0], width=width)
    
    return image  # 更新された画像を返す

# noは表示番号
no = 1
# 画像を読み込む

image_path = os.path.join(directory, "images", dataset.annotations[no -1]["cam/image_array"])
image = PIL.Image.open(image_path)
image_width, image_height = image.size

# HTMLウィジェットを作成
html_widget = ipywidgets.HTML(
    value=f"""
    <p>{image_path}</p>
    """
)
# 画像のバイトデータを取得
image_byte_arr = io.BytesIO()
image.save(image_byte_arr, format='JPEG')

# 画像を表示するウィジェットを作成
image_widget = ipywidgets.Image(
    value=image_byte_arr.getvalue(),
    format='jpg',
    width=image_width,
    height=image_height
)

# フォントを設定
font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 50)
# 512x512の白い画像を生成
bg_color="white"
font_color="black"
steering_image = Image.new('RGB', (control_image_width, control_image_height), bg_color)
throttle_image = Image.new('RGB', (control_image_width, control_image_height), bg_color)
# クロスラインを描く
steering_image = draw_cross_line(steering_image, (control_image_width//2,control_image_height//2), color=((191,191,191),(191,191,191)))
throttle_image = draw_cross_line(throttle_image, (control_image_width//2,control_image_height//2), color=((191,191,191),(191,191,191)))
# 文字を描く
steering_image = draw_font(steering_image, "Steering", font, font_color)
throttle_image = draw_font(throttle_image, "Throttle", font, font_color)


# 画像のバイトデータを取得
steering_image_byte_arr = io.BytesIO()
steering_image.save(steering_image_byte_arr, format='JPEG')
throttle_image_byte_arr = io.BytesIO()
throttle_image.save(throttle_image_byte_arr, format='JPEG')
steering_image_widget = ipywidgets.Image(
    value=steering_image_byte_arr.getvalue(),
    format='jpg',
    layout=ipywidgets.Layout(border='2px solid black')
)
throttle_image_widget = ipywidgets.Image(
    value=throttle_image_byte_arr.getvalue(),
    format='jpg',
    layout=ipywidgets.Layout(border='2px solid black')
)


# Eventインスタンスを作成
steering_event_handler = Event(source=steering_image_widget, watched_events=['click'])
throttle_event_handler = Event(source=throttle_image_widget, watched_events=['click'])
image_event_handler = Event(source=image_widget, watched_events=['click'])


# クリックイベントのハンドラ関数を定義
def steering_handle_event(event):
    coordinates = (event['offsetX'], event['offsetY'])
    write_log(f'steering coordinates: {coordinates}')
    
    # 元の画像データを読み込み
    steering_image_byte_arr.seek(0)
    image_copy = Image.open(steering_image_byte_arr).copy()
    
    # throttle座標はrecord_infoから取得する
    no = no_widget.value
    record_info = dataset.annotations[no - 1]
    write_log(f"{record_info}")
    x = 0
    y = record_info["user/throttle"]
    (x, y) = joystick_xy_to_image_coords(image_copy.size, x, y)
    coordinates = (event['offsetX'], y)
    
    # 直線を描く
    updated_image = draw_st_cross_line(image_copy, coordinates, color=((0,255,0),(223,255,223)))

    # 画像のバイトデータを再度取得
    updated_image_byte_arr = io.BytesIO()
    updated_image.save(updated_image_byte_arr, format='JPEG')

    # 画像を表示するウィジェットを更新
    steering_image_widget.value = updated_image_byte_arr.getvalue()
    
    # データを更新する
    no = no_widget.value
    record_info = dataset.annotations[no - 1]
    index = record_info["_index"]
    x = remap(coordinates[0], 0, updated_image.size[0]-1, -1.0, 1.0)
    x = max(min(x, 1.0), -1.0)
    record_info["user/angle"] = x
    dataset.save_record(index, record_info)
    load_img(no)

def throttle_handle_event(event):
    coordinates = (event['offsetX'], event['offsetY'])
    write_log(f'throttle coordinates: {coordinates}')
    
    # 元の画像データを読み込み
    throttle_image_byte_arr.seek(0)
    image_copy = Image.open(throttle_image_byte_arr).copy()
    
    # steering座標はrecord_infoから取得する
    no = no_widget.value
    record_info = dataset.annotations[no - 1]
    x = record_info["user/angle"]
    y = 0
    (x, y) = joystick_xy_to_image_coords(image_copy.size, x, y)
    coordinates = (x, event['offsetY'])
    
    # 直線を描く (ブレーキのときは赤色にする)
    if coordinates[1] <= image_copy.size[1] / 2:
        updated_image = draw_cross_line(image_copy, coordinates, color=((223,255,223),(0,255,0)))
    else:
        updated_image = draw_cross_line(image_copy, coordinates, color=((223,255,223),(255,0,0)))

    
    # 画像のバイトデータを再度取得
    updated_image_byte_arr = io.BytesIO()
    updated_image.save(updated_image_byte_arr, format='JPEG')
    
    # 画像を表示するウィジェットを更新
    throttle_image_widget.value = updated_image_byte_arr.getvalue()

    # データを更新する
    no = no_widget.value
    record_info = dataset.annotations[no - 1]
    index = record_info["_index"]
    y = remap(coordinates[1], updated_image.size[1]-1, 0, -1.0, 1.0)  # y軸は0が上=速度最大、真ん中が速度0、下がブレーキ最大
    y = max(min(y, 1.0), -1.0)
    record_info["user/throttle"] = y
    dataset.save_record(index, record_info)
    load_img(no)

def image_handle_event(event):
    image_coordinates = (event['offsetX'], event['offsetY'])
    write_log(f'image coordinates: {image_coordinates}')
    
    # 元の画像データを読み込み
    steering_image_byte_arr.seek(0)
    steering_image_copy = Image.open(steering_image_byte_arr).copy()
    image_byte_arr.seek(0)
    image_copy = Image.open(image_byte_arr).copy()
    
    # throttle座標はrecord_infoから取得する
    no = no_widget.value
    record_info = dataset.annotations[no - 1]
    write_log(f"{record_info}")
    x = 0
    y = record_info["user/throttle"]
    (x, y) = joystick_xy_to_image_coords(image_copy.size, x, y)  # レコードデータのJoystick座標をカメラ画像widgetのピクセル座標に変換
    image_coordinates = (event['offsetX'], y)  # x軸はマウスクリックのピクセル座標をつかい、y軸はレコードデータを変換した座標を使う

    # カメラ画像の座標をステアリング画像の座標に変更する
    x = remap(image_coordinates[0], 0, image_copy.size[0]-1, 0, steering_image_copy.size[0]-1)
    y = remap(image_coordinates[1], 0, image_copy.size[1]-1, 0, steering_image_copy.size[1]-1)
    coordinates = (int(x), int(y))

    # 直線を描く
    updated_image = draw_st_cross_line(steering_image_copy, coordinates, color=((0,255,0),(223,255,223)))

    # 画像のバイトデータを再度取得
    updated_image_byte_arr = io.BytesIO()
    updated_image.save(updated_image_byte_arr, format='JPEG')

    # 画像を表示するウィジェットを更新
    steering_image_widget.value = updated_image_byte_arr.getvalue()
    
    # データを更新する
    no = no_widget.value
    record_info = dataset.annotations[no - 1]
    index = record_info["_index"]
    x = remap(coordinates[0], 0, updated_image.size[0]-1, -1.0, 1.0)
    x = max(min(x, 1.0), -1.0)
    record_info["user/angle"] = x
    dataset.save_record(index, record_info)
    load_img(no)


# イベントハンドラを設定
steering_event_handler.on_dom_event(steering_handle_event)
throttle_event_handler.on_dom_event(throttle_handle_event)
image_event_handler.on_dom_event(image_handle_event)

def joystick_xy_to_image_coords(image_size, x, y):
    coord_x = remap(x, -1.0, 1.0, 0, image_size[0]-1)
    coord_y = remap(y, -1.0, 1.0, image_size[1]-1, 0)  # y軸は0が上=速度最大、真ん中が速度0、下がブレーキ最大
    coordinates = (coord_x, coord_y)
    return coordinates

def draw_steering_cross_line(x, y):
    # 元の画像データを読み込み
    steering_image_byte_arr.seek(0)
    image_copy = Image.open(steering_image_byte_arr).copy()
    coordinates = joystick_xy_to_image_coords(image_copy.size, x, y)
    
    # 直線を描く
    updated_image = draw_st_cross_line(image_copy, coordinates, color=((0,255,0),(223,255,223)))
    
    # 画像のバイトデータを再度取得
    updated_image_byte_arr = io.BytesIO()
    updated_image.save(updated_image_byte_arr, format='JPEG')
    
    # 画像を表示するウィジェットを更新
    steering_image_widget.value = updated_image_byte_arr.getvalue()

def draw_throttle_cross_line(x, y):
    # 元の画像データを読み込み
    throttle_image_byte_arr.seek(0)
    image_copy = Image.open(throttle_image_byte_arr).copy()
    coordinates = joystick_xy_to_image_coords(image_copy.size, x, y)
    
    # 直線を描く (ブレーキのときは赤色にする)
    if coordinates[1] <= image_copy.size[1] / 2:
        updated_image = draw_cross_line(image_copy, coordinates, color=((223,255,223),(0,255,0)))
    else:
        updated_image = draw_cross_line(image_copy, coordinates, color=((223,255,223),(255,0,0)))

    # 画像のバイトデータを再度取得
    updated_image_byte_arr = io.BytesIO()
    updated_image.save(updated_image_byte_arr, format='JPEG')
    
    # 画像を表示するウィジェットを更新
    throttle_image_widget.value = updated_image_byte_arr.getvalue()


# 汎用レイアウトを定義
description_style = {'description_width': 'initial'}
widget_width = ipywidgets.Layout(width=str(image.width)+'px')
widget_width_half = ipywidgets.Layout(width=str(image.width//2)+'px')
widget_width_third = ipywidgets.Layout(width=str(image.width//3)+'px')


########################################
# ボタンウィジェットのサイズを定義します。
########################################
prev_pic_button = ipywidgets.Button(description='prev', layout=widget_width_third)
next_pic_button = ipywidgets.Button(description='next', layout=widget_width_third)
delete_pic_button = ipywidgets.Button(description='delete', layout=widget_width_third)
play_pic_button = ipywidgets.Button(description='play', layout=widget_width_third)
make_movie_button = ipywidgets.Button(description='make movie', layout=widget_width)


def pred(image):
    # PILイメージをNumPy配列に変換
    image_np = np.array(image)

    # NumPy配列を浮動小数点型に変換
    image_np = image_np.astype('float32')
    # 0.0 - 1.0に変換
    # 推論を実行
    output = pilot.run(image_np)
    return output

def load_img(no):
    global image_byte_arr
    """
    noは1からn番までの値で、表示番号を表します。
    """
    if len(dataset) == 0:
        no_widget.value = 0
        write_log("データファイルが存在しません。")
        return
    if no > len(dataset):
        no = 1
    if no < 1:
        no = len(dataset)

    no_widget.value = no
    record_info = dataset.annotations[no - 1]
    image_path = os.path.join(directory, "images", record_info["cam/image_array"])
    write_log(str(no) + "枚目の" + image_path + "を読込みます。")
    

    x = record_info["user/angle"]
    y = record_info["user/throttle"]

    # -1.0から1.0の範囲に制限する
    x = max(min(x, 1.0), -1.0)
    y = max(min(y, 1.0), -1.0)

    write_log(f'{record_info}')
    
    # PILで画像を読み込む
    img = Image.open(image_path)
    marked_img = img.copy()

    # 推論を実行する
    pred_img = img.copy()
    (pilot_x, pilot_y) = pred(pred_img)
    write_log(f'pilot: ({pilot_x}, {pilot_y})')
    pilot_x = max(min(pilot_x, 1.0), -1.0)
    pilot_y = max(min(pilot_y, 1.0), -1.0)
 
    
    #coord_x = remap(x, -1.0, 1.0, 0, img.size[0]-1)
    #coord_y = remap(y, -1.0, 1.0, img.size[1]-1, 0)  # y軸は0が上=速度最大、真ん中が速度0、下がブレーキ最大

    coord_x = remap(pilot_x, -1.0, 1.0, 0, img.size[0]-1)
    coord_y = remap(pilot_y, -1.0, 1.0, img.size[1]-1, 0)  # y軸は0が上=速度最大、真ん中が速度0、下がブレーキ最大

    #write_log(f"({x:.2f},{y:.2f}), ({coord_x},{coord_y}), {image_path}")
    write_log(f"({pilot_x:.2f},{pilot_y:.2f}), ({coord_x},{coord_y}), {image_path}")
    # PILのImageDrawを使用して画像に円を描画
    draw = ImageDraw.Draw(marked_img)
    r = 4
    # 直線を描く (ブレーキのときは赤色にする)
    if y >= 0:
        draw.ellipse((coord_x-r, coord_y-r, coord_x+r, coord_y+r), fill=(0, 255, 0))
    else:
        draw.ellipse((coord_x-r, coord_y-r, coord_x+r, coord_y+r), fill=(255, 0, 0))
    
    # 画像をJPEG形式のバイトデータに変換
    image_byte_arr = io.BytesIO()
    marked_img.save(image_byte_arr, format='JPEG')
    
    image_widget.value = image_byte_arr.getvalue()
    write_log(str(no) + "枚目の" + image_path + f"を読込みました。({x}, {y})")
    html_widget.value=f"""
    <p>{image_path}</p>
    """
    #draw_steering_cross_line(x, y)
    #draw_throttle_cross_line(x, y)
    draw_steering_cross_line(pilot_x, pilot_y)
    draw_throttle_cross_line(pilot_x, pilot_y)

def delete_img(no):
    """
    load_img()との違いは、最後のファイルを削除したときに、先頭のファイルを表示するのではなく、最後尾のファイルを表示すること。
        if no > len(dataset):
        no += -1
    noは1からn番までの値で、ファイル番号を表します。
    """
    is_last_index = False
    if len(dataset) == 0:
        no_widget.value = 0
        write_log("データファイルが存在しません。")
        return
    if no >= len(dataset):
        no = len(dataset)
    if no < 1:
        no = 1
    
    if no == len(dataset):
        is_last_index = True

    record_info = dataset.annotations[no - 1]
    image_path = os.path.join(directory, "images", record_info["cam/image_array"])

    write_log(str(no) + "枚目の" + image_path + "を削除します。")
    dataset.delete_record(no - 1)

    # ファイル最後尾を削除した場合、新しい最後尾のファイルを表示する
    if is_last_index:
        no = len(dataset)
    no_widget.value = no
    load_img(no)


def prev_pic(c):
    stop_play()
    no = no_widget.value
    no = int(no) - 1
    load_img(no)

def next_pic(c):
    stop_play()
    no = no_widget.value
    no = int(no) + 1
    load_img(no)
    no = no_widget.value

def delete_pic(c):
    stop_play()
    no = no_widget.value
    delete_img(no)

def stop_play():
    global is_play
    play_pic_button.description = "start"
    if is_play:
        is_play = False
        write_log("play stop")
    else:
        write_log("already stopped")

def play():
    global is_play
    no = no_widget.value
    try:
        while is_play:
            no = int(no) + 1
            if len(dataset) == 0:
                no_widget.value = 0
                write_log("データファイルが存在しません。")
                stop_play()
                load_img(no)
                break
            elif no > len(dataset):
                no = 1
            load_img(no)
            time.sleep(0.01) # 100Hz
    finally:
        pass

def play_pic(c):
    global is_play, play_thread
    is_play ^= True
    if is_play:
        if play_thread is None or not play_thread.is_alive():
            play_thread = threading.Thread(target=play)
            play_thread.start()
            play_pic_button.description = "stop"
            play_pic_button.style.button_color = 'orange'
            write_log("play start")
        else:
            write_log("already started")
    else:
        write_log("play stop")
        play_pic_button.description = "start"
        play_pic_button.style.button_color = None
    return


def combine_images_opencv(img1, img2, img3, gap=10):
    h = max(img1.shape[0], img2.shape[0], img3.shape[0])
    total_width = img1.shape[1] + img2.shape[1] + img3.shape[1] + 2 * gap

    combined_image = np.zeros((h, total_width, 3), dtype=np.uint8)
    combined_image.fill(255)  # 白背景

    combined_image[:img1.shape[0], :img1.shape[1]] = img1
    combined_image[:img2.shape[0], img1.shape[1]+gap:img1.shape[1]+gap+img2.shape[1]] = img2
    combined_image[:img3.shape[0], img1.shape[1]+img2.shape[1]+2*gap:] = img3

    return combined_image


def get_image_size_from_widget(widget):
    """ウィジェットから画像のサイズを取得する関数"""
    img = cv2.imdecode(np.frombuffer(widget.value, np.uint8), cv2.IMREAD_COLOR)
    return img.shape[1], img.shape[0]  # 幅と高さ

def calculate_combined_size(img1_size, img2_size, img3_size, gap=10):
    """3つの画像サイズを組み合わせて、結合後のサイズを計算する関数"""
    width = img1_size[0] + img2_size[0] + img3_size[0] + 2 * gap
    height = max(img1_size[1], img2_size[1], img3_size[1])
    return width, height

def make_movie_opencv():
    global is_movie
    if len(dataset) == 0:
        write_log("データファイルが存在しません。")
        is_movie = False
        return

    # 最初のフレームを使用して結合後のサイズを計算
    img1_size = get_image_size_from_widget(image_widget)
    img2_size = get_image_size_from_widget(steering_image_widget)
    img3_size = get_image_size_from_widget(throttle_image_widget)
    combined_size = calculate_combined_size(img1_size, img2_size, img3_size)

    # 動画の設定
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('predict_video.mp4', fourcc, 30.0, combined_size)
    try:
        for i in range(1, len(dataset) + 1):
            if not is_movie:
                break
            # 画像を取得
            img1 = cv2.imdecode(np.frombuffer(image_widget.value, np.uint8), cv2.IMREAD_COLOR)
            img2 = cv2.imdecode(np.frombuffer(steering_image_widget.value, np.uint8), cv2.IMREAD_COLOR)
            img3 = cv2.imdecode(np.frombuffer(throttle_image_widget.value, np.uint8), cv2.IMREAD_COLOR)

            # 画像を結合
            combined_image = combine_images_opencv(img1, img2, img3)

            # 動画にフレームを追加
            out.write(combined_image)
            next_pic(i)

        out.release()
    finally:
        is_movie = False

def make_movie(c):
    global is_movie, movie_thread
    is_movie ^= True
    if is_movie:
        if movie_thread is None or not movie_thread.is_alive():
            movie_thread = threading.Thread(target=make_movie_opencv)
            movie_thread.start()
            make_movie_button.description = "stop"
            make_movie_button.style.button_color = 'orange'
            write_log("make movie start")
        else:
            write_log("already started")
    else:
        write_log("make movie stop")
        make_movie_button.description = "make movie"
        make_movie_button.style.button_color = None
    return

play_thread = None
movie_thread = None
is_play = False
is_movie = False
prev_pic_button.on_click(prev_pic)
next_pic_button.on_click(next_pic)
delete_pic_button.on_click(delete_pic)
play_pic_button.on_click(play_pic)
make_movie_button.on_click(make_movie)

no_widget = ipywidgets.IntText(description='no', style=description_style, layout=widget_width, value=no)

def on_no_widget_change(change):
    new_value = change['new']
    load_img(new_value)

no_widget.observe(on_no_widget_change, names='value')

load_img(1)

# 画像ウィジェットを表示
display(ipywidgets.VBox([
    html_widget,
    ipywidgets.HBox([
        ipywidgets.VBox([
            image_widget,
            no_widget,
            ipywidgets.HBox([play_pic_button, prev_pic_button, delete_pic_button, next_pic_button]),
            make_movie_button,
        ], layout=ipywidgets.Layout(align_items='center')),
        steering_image_widget,
        throttle_image_widget
    ])
]))

display(write_log)

In [ ]:
import json
import os
import re

def generate_catalog_manifest(catalog_path):
    # 各行の長さを格納するリストを初期化
    line_lengths = []

    # catalogファイルを読み込んで各行の長さを計算
    with open(catalog_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 各行の長さ（バイト数）を計算（改行文字も含む）
            line_length = len(line.encode('utf-8'))
            line_lengths.append(line_length)

    # manifestファイルのパスを生成
    # すでに ".catalog_manifest" が含まれている場合は追加しない
    manifest_path = catalog_path + "_manifest" if not catalog_path.endswith("_manifest") else catalog_path

    # manifestデータを作成
    manifest_data = {
        "created_at": os.path.getctime(catalog_path),
        "line_lengths": line_lengths,
        "path": os.path.basename(manifest_path),
        "start_index": 0
    }

    # manifestファイルを書き込む
    with open(manifest_path, 'w', encoding='utf-8') as manifest_file:
        json.dump(manifest_data, manifest_file)

def process_all_catalogs(directory):
    # ディレクトリ内のすべてのcatalogファイルを探す
    catalog_files = [f for f in os.listdir(directory) if re.match(r'catalog_\d+\.catalog', f)]

    for catalog_file in catalog_files:
        catalog_path = os.path.join(directory, catalog_file)
        manifest_path = catalog_path + "_manifest"
        generate_catalog_manifest(catalog_path)

# 実行例
process_all_catalogs(directory)
